In [ ]:
import pyemu
import os
import matplotlib.pyplot as plt

## run NOPTMAX=0 to get residuals, then load and reweight

In [ ]:
input_dir = '../noptmax0_testing/'
output_dir = '../run_data'
pst_root = 'prior_mc_wide'

In [ ]:
pst = pyemu.Pst(os.path.join(input_dir,'{}.pst'.format(pst_root)))

In [ ]:
pst.plot(kind='phi_pie')
plt.savefig('../report_materials/prior_mc_wide_phi_pie.pdf', dpi=600)

In [ ]:
new_proportions = pst.phi_components.copy()
new_proportions

In [ ]:
pst.observation_data

In [ ]:
new_proportions['flux'] = 0.3*pst.phi
new_proportions['head'] = 0.2*pst.phi
new_proportions['land_surface'] = 0.5*pst.phi
new_proportions['budget'] = 0

In [ ]:
new_proportions

In [ ]:
pst.adjust_weights(obsgrp_dict=new_proportions)

In [ ]:
pst.observation_data

In [ ]:
pst.plot(kind='phi_pie')
plt.savefig('../report_materials/prior_mc_wide_REWEIGHTED_phi_pie.pdf', dpi=600)

In [ ]:
# set some values for ies
pst.pestpp_options["ies_num_reals"] = 500  
pst.pestpp_options["ies_bad_phi_sigma"] = 2.5
pst.pestpp_options["overdue_giveup_fac"] = 4
pst.pestpp_options["ies_save_rescov"] = True
pst.pestpp_options["ies_no_noise"] = True
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["ies_pdc_sigma_distance"] = 3.0
pst.pestpp_options['ies_autoadaloc']=True
pst.pestpp_options['ies_num_threads']=4
pst.pestpp_options['ies_lambda_mults']=(0.1,1.0,10.0,100.0)
pst.pestpp_options['lambda_scale_fac'] = (0.75,0.9,1.0,1.1)
pst.pestpp_options['ies_subset_size']=10

# set SVD for some regularization
pst.svd_data.maxsing = 250

In [ ]:
# set up for prior Monte Carlo
pst.control_data.noptmax = -1
pst.write(os.path.join(output_dir, '{}.pst'.format(pst_root)))